In [1]:
# Cell 1: Imports and Setup
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import openai

In [2]:
# Cell 2: Load Environment Variables and Verify API Key
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please set the OPENAI_API_KEY in your .env file.")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start with 'sk-proj-'; please check you're using the right key.")
elif api_key.strip() != api_key:
    print("An API key was found, but it has leading or trailing whitespace - please remove them.")
else:
    print("API key found and looks good so far!")
    openai.api_key = api_key  # Set the API key for OpenAI

API key found and looks good so far!


In [3]:
# Cell 3: Define Headers for HTTP Requests
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/117.0.0.0 Safari/537.36"
    )
}

In [4]:
# Cell 4: Define the Website Class
class Website:
    def __init__(self, url):
        """
        Create a Website object from the given URL using BeautifulSoup.
        """
        self.url = url
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise error if status is not 200
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {e}")
            self.title = "Error fetching the website"
            self.text = ""
            return

        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string.strip() if soup.title and soup.title.string else "No title found"

        # Remove irrelevant tags
        for tag in soup.find_all(["script", "style", "img", "input", "nav", "footer", "header", "aside"]):
            tag.decompose()

        # Get text from the body, if available
        body = soup.body
        if body:
            self.text = body.get_text(separator="\n", strip=True)
        else:
            self.text = "No body content found."

In [12]:
# Cell 5: Define System Prompt
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary in german, ignoring text that might be navigation related. "
    "Respond in markdown."
)

In [13]:
# Cell 6: Define User Prompt Function
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled '{website.title}'.\n"
    user_prompt += (
        "The contents of this website are as follows:\n"
        "Please provide a short summary of this website in markdown. "
        "If it includes news or announcements, then summarize these too.\n\n"
    )
    user_prompt += website.text
    return user_prompt

In [14]:
# Cell 7: Define Messages for OpenAI API
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [15]:
# Cell 8: Define Summarization Function
def summarize(url):
    website = Website(url)
    if not website.text:
        return f"Could not retrieve content from {url}."

    try:
        # Use the available ChatCompletion API (choose a model available in your account, e.g., "gpt-4" or "gpt-3.5-turbo")
        response = openai.chat.completions.create(
            model="gpt-4o-mini",  # Change as necessary
            messages=messages_for(website)
        )
        summary = response.choices[0].message.content.strip()
        return summary
    except Exception as e:
        # Catching all exceptions instead of openai.error.OpenAIError since your module might not expose that attribute
        return f"An error occurred while summarizing {url}: {e}"

In [16]:
# Cell 9: Define Display Summary Function
def display_summary(url):
    print(f"Fetching and summarizing: {url}")
    summary = summarize(url)
    if summary:
        display(Markdown(f"### Summary for [{url}]({url})\n\n{summary}"))
    print("\n" + "="*80 + "\n")

In [19]:
# Cell 10: Define List of URLs to Summarize
urls = [
    "https://www.duesseldorp.de",
    "https://www.github.com/Duesseldorp",
    # Add more URLs as needed
]

In [20]:
# Cell 11: Iterate Over URLs and Display Summaries
for url in urls:
    try:
        display_summary(url)
    except Exception as e:
        print(f"Failed to summarize {url}: {e}")
        print("\n" + "="*80 + "\n")

Fetching and summarizing: https://www.duesseldorp.de


### Summary for [https://www.duesseldorp.de](https://www.duesseldorp.de)

# Zusammenfassung der Website 'DUESSELDORP'

Die Website 'DUESSELDORP' kombiniert Kunst, Fitness, Künstliche Intelligenz (KI) und Blockchain-Technologie und feiert die vielfältige Kulturszene in Düsseldorf, einschließlich des Karnevals. Der Blog-Autor teilt seine Leidenschaft für digitale Trends, Kunst im öffentlichen Raum und die lebendige Atmosphäre der Stadt.

## Hauptinhalte

- **Düsseldorfer Karneval 2025**: Ein umfassender Leitfaden zu den besten Events und Highlights des Karnevals in Düsseldorf, inklusive Insider-Tipps.
  
- **Künstliche Intelligenz**: Artikel über die Fusion von KI, Kunst und Musik sowie die Möglichkeiten, die KI für kreative Innovationen bietet.

- **Blockchain und DLT**: Informationen über die Bedeutung von Blockchain-Technologien und deren Anwendung zur Verbesserung von Effizienz und Transparenz.

## Aktuelle Artikel

- **Ollama für Windows**: Vorstellung einer Open-Source-KI-Plattform zur einfachen und sicheren Einrichtung lokaler KI-Modelle.

- **AGI: Chancen und Risiken**: Diskussion über die bevorstehenden Entwicklungen und Herausforderungen im Zusammenhang mit Künstlicher Allgemeiner Intelligenz.

- **Mensch, Maschine und das Echo der Zukunft**: Ein Einblick in die Bedeutung automatisierter Prozesse im modernen Leben.

Die Website ermutigt die Leser, sich mit den Themen Technik, Kunst und der Düsseldorfer Kultur auseinanderzusetzen und die künstlerischen sowie betrieblichen Möglichkeiten der digitalen Revolution zu erkunden.



Fetching and summarizing: https://www.github.com/Duesseldorp


### Summary for [https://www.github.com/Duesseldorp](https://www.github.com/Duesseldorp)

```markdown
### Zusammenfassung der Website 'Duesseldorp (Martin Gräbing) · GitHub'

Die Website gehört Martin Gräbing und ist auf GitHub gehostet. Sie ist als Portfolio für das Projekt 'Duesseldorp' gestaltet, dass vermutlich im Bereich der Künstlichen Intelligenz liegt. Ein relevantes Projekt, das erwähnt wird, ist 'ai_experimental', welches als öffentliches Experimentierfeld für KI in Jupyter Notebooks dient.

Zusätzlich enthält die Seite Links zu anderen Funktionen von GitHub, wie Repositories, Projekten und Paketen, sowie Informationen über die Mitgliedschaft seit dem 10. Januar 2025.
```